# 1 Libraries

In [1]:
from influxdb import DataFrameClient
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
import requests 
from datetime import datetime, timedelta
import time
import ast

# 2 Order Book merge functions

# 2.1 Binance

In [2]:
# EVENT_TIME (E)
def create_Binance(part, part_dif, last_row):
    # Pass numbers to float format
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Seg"].apply(lambda x: ast.literal_eval(x))
    part.loc[:, "Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:, "Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: dict(list([[i[0],float(i[1])] for i in ast.literal_eval(x)])))

    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"
    if type(last_row) == type(None):
        # Gather first snapshot valid
        first = 0
        for i in range(len(part)):
            for j in range(len(part_dif)):
                if int(part_dif.iloc[j,4][1]) <= int(part.iloc[i,3]) and int(part_dif.iloc[j,4][0]) > int(part.iloc[i,3]):
                    first = [i,j]
                    break
                elif int(part_dif.iloc[j,4][1]) > int(part.iloc[i,3]):
                    break
            if first != 0:
                break

    else:
        first = [0,0] 

    # Add rest of snapshots to the dataset
    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""

    firsts = [element[1] for element in part_dif.iloc[1::,4]]
    seconds = [element[0] for element in part_dif.iloc[1::,4]]

    pos = []

    df = part
    for element in df.iloc[:,3]:
        pos.append(next(x[0] for x in enumerate(firsts) if x[1] >=  int(element)))
        i = 0
        for element in pos:
            if seconds[pos[i]] >= int(df.iloc[i,3]):
                part_dif.iat[element,6] = df.iloc[i,0]
                part_dif.iat[element,7] = df.iloc[i,1]
            i += 1
    # Rebuild order book for each timestemp we have

    # Start from first snapshot
    part_join = part_dif.iloc[first[1]::,[0,1,4,5,6,7]]
    if type(last_row) != type(None):
        part_join = pd.concat([pd.DataFrame(last_row).T, part_join])

    # Rebuild with differences, keeping snapshots when possible
    for i in range(len(part_join)):
        for j in range(2):
            if part_join.iloc[i,4+j] == "":
                part_join.iat[i,4+j] = part_join.iloc[i-1,4+j].copy()
                if i > 1:
                    if part_join.iloc[i,2][1] != part_join.iloc[i-1,2][0] + 1:
                        print("\n BINANCE ERROR")
                        print("Missing records between " + str(part_join.iloc[i-1,2][0]) + " and " + str(part_join.iloc[i,2][1]) + "\n")
            for key in part_join.iloc[i,j].keys():
                if  key in part_join.iloc[i,4+j]:
                    if part_join.iloc[i,j][key] == 0.0:
                        del part_join.iloc[i,4+j][key]
                    else:
                        part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
                elif part_join.iloc[i,j][key] != 0.0:
                    part_join.iat[i,4+j][key] = part_join.iloc[i,j][key]
    lrow = part_join.iloc[-1,:]     

    return part_join.iloc[1::,:], lrow

# 2.2 Bitfinex

In [3]:
def create_Bitfinex(part, part_dif, last_row):
    
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[2]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[2]] for i in ast.literal_eval(x)]) if x != "-" else x)  
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: float(x)) 
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: float(x)) 
    part_dif.loc[:,"Seg"] = part_dif.loc[:,"Bids"].apply(lambda x: int(x)) 

    join_part = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair","Seg"]]]).sort_index()
    if type(last_row) != type(None):
        join_part = pd.concat([pd.DataFrame(last_row).T, join_part])
    join_part["U_Asks"] = ""
    join_part["U_Bids"] = ""
    join_part.iat[0,5] = part.iloc[0,0].copy()
    join_part.iat[0,6] = part.iloc[0,1].copy()
    
    current = join_part.iloc[0,3]
    for i in range(1,len(join_part)):
        join_part.iat[i,5] = join_part.iloc[i-1,5].copy()
        join_part.iat[i,6] = join_part.iloc[i-1,6].copy()
        if type(join_part.iloc[i,1]) != type({}) and join_part.iloc[i-1,3] == current:
            if join_part.iloc[i,0] < 0:
                if  join_part.iloc[i,1] in join_part.iloc[i,5]:
                    if join_part.iloc[i,4] == 0 and join_part.iloc[i,0] == -1:
                        del join_part.iloc[i,5][join_part.iloc[i,1]]
                    else:
                        join_part.iat[i,5][join_part.iloc[i,1]] = float(join_part.iloc[i,0])
                else:
                    join_part.iat[i,5][join_part.iloc[i,1]] = join_part.iloc[i,0]
            if join_part.iloc[i,0] > 0:
                if  join_part.iloc[i,1] in join_part.iloc[i,6] and join_part.iloc[i,0] == 1:
                    if join_part.iloc[i,4] == 0:
                        del join_part.iloc[i,6][join_part.iloc[i,1]]
                    else:
                        join_part.iat[i,6][join_part.iloc[i,1]] = join_part.iloc[i,0]
                else:
                    join_part.iat[i,6][join_part.iloc[i,1]] = join_part.iloc[i,0]
        elif type(join_part.iloc[i,1]) == type({}):
            join_part.iat[i,5] = join_part.iloc[i,0].copy()
            join_part.iat[i,6] = join_part.iloc[i,1].copy()
            current = join_part.iloc[0,3]
        else:
            join_part.iat[i,5] = join_part.iloc[i-1,5].copy()
            join_part.iat[i,6] = join_part.iloc[i-1,6].copy()
    
    join_part["U_Asks"] = join_part["U_Asks"].apply(lambda x: dict([[str(key), np.abs(x[key])] for key in x.keys()]))
    lrow = join_part.iloc[-1,:]
    return join_part.iloc[1::,:], lrow

# 2.3 Bithumb

In [4]:
# Timestamp in msg
def create_Bithumb(part, part_dif, last_row):
    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x:[[element[0],float(element[1])] for element in ast.literal_eval(x)] if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: [[element[0],float(element[1])] for element in ast.literal_eval(x)] if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x: dict([[element[0],float(element[1])] for element in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict([[element[0],float(element[1])] for element in ast.literal_eval(x)]) if x != "-" else x)
    part["Type"] = "Snapshot"
    part_dif["Type"] = "Dif"    
    
    # Create updated order book
    if len(part) != 0:  
        part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair", "Type"]], part_dif[["Asks","Bids", "Host", "Pair", "Type"]]]).sort_index()
    else:
        part_dif = part_dif[["Asks","Bids", "Host", "Pair", "Type"]]
    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])  
        part_dif.iat[0,4] = "Previous"   
        
    for i in range(len(part_dif)):
        if part_dif.iloc[i,4] == "Snapshot":
            part_dif.iat[i,5] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,6] = part_dif.iloc[i,1].copy()
        elif part_dif.iloc[i,4] == "Dif":            
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
            part_dif.iat[i,6] = part_dif.iloc[i-1,6].copy()
            for element in part_dif.iat[i,0]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,5] and element[1] == '0.000000':
                        del part_dif.iloc[i,5][element[0]]
                    else:
                        part_dif.iat[i,5][element[0]] = element[1]
            for element in part_dif.iat[i,1]:
                if element != "[]":
                    if element[0] in part_dif.iloc[i,6] and element[1] == '0.000000':
                        del part_dif.iloc[i,6][element[0]]
                    else:
                        part_dif.iat[i,6][element[0]] = element[1]
    lrow = part_dif.iloc[-1,:] 
    return part_dif.iloc[1::,:], lrow

# 2.4 Bitstamp

In [5]:
# Timestamp in message
def create_Bitstamp(part, part_dif):
    # First updates

    # Evaluate strings
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    
    # Create updated order book

    part_join = part.rename(columns = {"Asks":"U_Asks", "Bids":"U_Bids"})
    return part_join

# 2.5 Coinbase

In [6]:
# Timestamp in message
def create_Coinbase(part, part_dif, last_row):

    # Evaluate strings
    part_dif.loc[:,"Asks"] = part_dif.loc[:,"Asks"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part_dif.loc[:,"Bids"] = part_dif.loc[:,"Bids"].apply(lambda x: ast.literal_eval(x) if x != "-" else x)
    part.loc[:,"Asks"] =  part.loc[:,"Asks"].apply(lambda x:dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] =  part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    
    part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair"]]]).sort_index()
    
    part_dif = part_dif.loc[part_dif[part_dif["Asks"] != "-"].index[0]::,:]
    #   In case we want to change indexes
    # part_dif.index.values[i]
    # Rebuild order book for each timestemp we have

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,4] = part.iloc[0,0].copy()
    part_dif.iat[0,5] = part.iloc[0,1].copy()
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])      
    current = part_dif.iloc[0,3]
    for i in range(1,len(part_dif)):
        part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
        part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
        if part_dif.iloc[i,0] == "-" and part_dif.iloc[i-1,3] == current:
            for element in part_dif.iloc[i,1]:
                if  "sell" == element[0]:
                    if element[1] in part_dif.iloc[i,4]:
                        if element[2] == '0.00000000':
                            del part_dif.iloc[i,4][element[1]]
                        else:
                            part_dif.iat[i,4][element[1]] = element[2]
                    elif element[2] != '0.00000000':
                        part_dif.iat[i,4][element[1]] = element[2]
                else:
                    if element[1] in part_dif.iloc[i,5]:
                        if element[2] == '0.00000000':
                            del part_dif.iloc[i,5][element[1]]
                        else:
                            part_dif.iat[i,5][element[1]] = element[2]
                    elif element[2] != '0.00000000':
                        part_dif.iat[i,5][element[1]] = element[2]          
        elif part_dif.iloc[i,0] != "-":
            part_dif.iat[i,4] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,5] = part_dif.iloc[i,1].copy()
            current = part_dif.iloc[0,3]
        else:
            part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
    
    lrow = part_dif.iloc[-1,:]
    return part_dif.iloc[1::,:], lrow

# 2.6 Huobi

In [7]:
# Timestamp in message
def create_Huobi(part, part_dif):
    # First updates
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict(ast.literal_eval(x)) if x != "-" else x)  
    part = part.rename(columns = {"Asks": "U_Asks", "Bids": "U_Bids"})
    return part

# 2.7 Kraken

In [8]:
# Timestamp in message
# Careful with "r" for republished updates!
def create_Kraken(part, part_dif, last_row):
    part.at[:,"Host"] = "Snapshot"
    
    part_dif = pd.concat([part[["Asks","Bids", "Host", "Pair"]], part_dif[["Asks","Bids", "Host", "Pair"]]]).sort_index()
    part.loc[:,"Asks"] = part.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part.loc[:,"Bids"] = part.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)  
    part_dif.loc[:,"Asks"] =  part_dif.loc[:,"Asks"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)
    part_dif.loc[:,"Bids"] =  part_dif.loc[:,"Bids"].apply(lambda x: dict([[i[0],i[1]] for i in ast.literal_eval(x)]) if x != "-" else x)        # Rebuild order book for each timestemp we have

    part_dif = part_dif.loc[part_dif[part_dif["Host"] == "Snapshot"].index[0]::,:]

    part_dif["U_Asks"] = ""
    part_dif["U_Bids"] = ""
    part_dif.iat[0,4] = part.iloc[0,0].copy()
    part_dif.iat[0,5] = part.iloc[0,1].copy()
    if type(last_row) != type(None):
        part_dif = pd.concat([pd.DataFrame(last_row).T, part_dif])   

    current = part_dif.iloc[0,3]
    for i in range(1,len(part_dif)):
        if part_dif.iloc[i,2] != "Snapshot" and part_dif.iloc[i-1,3] == current:
            for j in range(2):
                part_dif.iat[i,4+j] = part_dif.iloc[i-1,4+j].copy()
                if part_dif.iloc[i,j] != "-":
                    for key in part_dif.iloc[i,j].keys():
                        if  key in part_dif.iloc[i,4+j]:
                            if float(part_dif.iloc[i,j][key][0]) == 0.0:
                                del part_dif.iloc[i,4+j][key]
                            else:
                                part_dif.iat[i,4+j][key] = part_dif.iloc[i,j][key][0]
                        else:
                            part_dif.iat[i,4+j][key] = part_dif.iloc[i,j][key][0]        
        elif part_dif.iloc[i,2] == "Snapshot":
            part_dif.iat[i,4] = part_dif.iloc[i,0].copy()
            part_dif.iat[i,5] = part_dif.iloc[i,1].copy()
            current = part_dif.iloc[0,3]
        else:
            part_dif.iat[i,4] = part_dif.iloc[i-1,4].copy()
            part_dif.iat[i,5] = part_dif.iloc[i-1,5].copy()
       
    lrow = part_dif.iloc[-1,:]
    
    return part_dif.iloc[1::,:], lrow

# 3.- General Functions

In [9]:
def compute_stats(ex, part_dif):
    # Compute orderbook price to add to the main dataset
    first = 0
    second = 0
    part_dif["OrderBookPrice " + ex] = ""
    indb = part_dif.columns.get_loc("U_Bids")
    inda = part_dif.columns.get_loc("U_Asks")
    indf = part_dif.columns.get_loc("OrderBookPrice " + ex) 
    for i in range(len(part_dif)):

        values = sorted(part_dif.iloc[i,inda].keys())
        values2 = sorted(part_dif.iloc[i,indb].keys(), reverse = True)
        for j in values:
            if float(part_dif.iloc[i,inda][j]) > 0:
                first = float(j)
                break
        for j in values2:
            if float(part_dif.iloc[i,indb][j]) > 0:
                second = float(j)
                break
        part_dif.iat[i,indf] = (first + second) / 2
        
        part_dif.iat[i,indb] = { your_key: part_dif.iat[i,indb][your_key] for your_key in values2[0:40] }
        part_dif.iat[i,inda] = { your_key: part_dif.iat[i,inda][your_key] for your_key in values[0:40] }

    separate = part_dif.iloc[:,[inda,indb]].apply(lambda x: pd.Series([item for sublist in  
                                                 [[float(list(x[0].keys())[i]), float(list(x[0].values())[i]),
                                                   float(list(x[1].keys())[i]), float(list(x[1].values())[i])] for i in range(40)] 
                                                  for item in sublist]), axis = 1).rename(columns = dict(
                             [[i,"Price_A_" + str(int(i/4)) + "_" + ex] if i % 4 == 0 else 
                             [i,"Price_B_" + str(int(i/4)) + "_" + ex] if i % 4 == 2  else
                             [i,"Quan_A_" + str(int(i/4)) + "_" + ex] if i % 4 == 1 else
                             [i,"Quan_B_" + str(int(i/4)) + "_" + ex] for i in range(160)]))
#     [print(col) for col in separate.columns]
    part_dif = pd.concat([part_dif.drop(["U_Asks", "U_Bids"], axis = 1), separate], axis = 1)
    
    return part_dif

In [10]:
def join_main(ex, df2, part_join):
    filter_col = [col for col in part_join if (col.startswith('Price_') or col.startswith('Quan_') or col.startswith('OrderBook'))]
    df2 = pd.merge(df2,part_join[filter_col], how='outer', left_index=True, right_index=True)
    filter_col = [col for col in df2 if (col.startswith('Last') or col.startswith('Order')
                                         or col.startswith("Price_") or col.startswith("Quan_"))]
    df2.update(df2[filter_col].ffill())
    filter_col = [col for col in df2 if (col.startswith('Q ') or col.startswith('Q_'))]
    df2.update(df2[filter_col].fillna(0))
    
    return df2

In [11]:
def create_OrderBook(ex, df2, part,part_dif, last_row = None):
    if ex == "Bitstamp":
        part_dif = create_Bitstamp(part, part_dif)
    elif ex == "Bitfinex":
        part_dif, last_row = create_Bitfinex(part, part_dif, last_row)
    elif ex == "Binance":
        part_dif, last_row = create_Binance(part, part_dif, last_row)
    elif ex == "Huobi":
        part_dif = create_Huobi(part, part_dif)
    elif ex == "Kraken":
        part_dif, last_row = create_Kraken(part, part_dif, last_row)
    elif ex == "Coinbase":
        part_dif, last_row = create_Coinbase(part, part_dif, last_row)
    elif ex == "Bithumb":
        part_dif, last_row = create_Bithumb(part, part_dif, last_row)
    
    part_dif = compute_stats(ex, part_dif)
    df2 = join_main(ex, df2,part_dif)
    
    return df2, last_row

In [12]:
def create_core(df, pair, exchanges, lrow):
    df1 = df[df["Pair"] == pair]
    
    df2 = df1[df1["Host"] == exchanges[0]].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
    df2.columns = df2.columns.droplevel(level = 0)
    df2 = df2.rename(columns = {"mean": "LastTrade_" + exchanges[0], "sum": "Q_" + exchanges[0]})
    for i in exchanges[1::]:
        dft = df1[df1["Host"] == i].groupby(pd.Grouper(freq = "ms")).agg({"Price":[np.mean],"Q":[np.sum]}).fillna(method = "ffill")
        df2[["LastTrade_" + i, "Q_"+i]] = dft[["Price", "Q"]]
        
    if type(lrow) != type(None):
        df2 = pd.concat([pd.DataFrame(lrow).T, df2])
        
    col1 = [col for col in df2 if (col.startswith('Last'))]
    df2.update(df2[col1].ffill().bfill())
    col2 = [col for col in df2 if (col.startswith('Q'))]
    df2.update(df2[col2].fillna(0))
    
    df2.drop_duplicates(keep = "first", inplace = True)
    
    return df2.iloc[1::,:]

In [13]:
def query(date_init, date_end):
    
    points = client.query("SELECT Host, Pair, Price, Q FROM Price WHERE time >= '"+ str(date_init) +"' AND time < '" + str(date_end) + "'")
    points_book = client.query("SELECT Asks,Bids, Host, LastUpdateID, Pair FROM Book WHERE time >= '"+ str(date_init) +"' AND time < '" + str(date_end) + "' AND LastUpdateID != ''")
    points_dif = client.query("SELECT Asks, Bids, Host, Pair, Seg FROM difBook WHERE time >= '"+ str(date_init) +"' AND time < '" + str(date_end) + "'")

    last_trades = points["Price"]
    book_snap = points_book["Book"]
    book_dif = points_dif["difBook"]
    
    return last_trades,book_snap,book_dif

In [14]:
def last_metrics_rmdups(df):
    # Compute the max difference and other metrics and remove duplicates
    filter_col = [col for col in df if col.startswith('Order')]
    df["Maximum"] = df[filter_col].apply(lambda row: np.max(row) - np.min(row), axis=1)

    df["Average"] = df[filter_col].apply(lambda row: np.mean(row)*0.002, axis=1)

    df["MaxExch"] = df[filter_col].fillna(0).apply(lambda row: sorted([row.idxmax()[15::], row.idxmin()[15::]])[0], axis=1)
    df["MinExch"] = df[filter_col].fillna(0).apply(lambda row: sorted([row.idxmax()[15::], row.idxmin()[15::]])[1], axis=1)

    return df

In [15]:
def calculate_labels(df):
    # If there are more than 10 consecutive >avg*0,002 then 1 otherwise 0.
    df["Index_Not"] = df[["Average","Maximum"]].apply(lambda x: 1 if x[0]<x[1] else 0, axis = 1)
    df["Index"] = df["Index_Not"]
    for i in range(len(df)-500):
        array = np.diff(df["Index_Not"][i:i+500])
        starts, = np.where(array>0) + np.ones(len(np.where(array>0)))
        ends, = np.where(array<0) + np.ones(len(np.where(array<0)))
        if len(starts) != len(ends):
            if len(ends) == 0:
                ends = np.append(ends,len(array))
            elif len(starts) == 0:
                starts = np.append(0,starts)
            elif starts[0] > ends[0]:
                starts = np.append(0,starts)
            else:
                ends = np.append(ends,len(array))

        if len(ends) != 0 and len(starts) != 0:
            if np.max(ends - starts) >= 10:
                df.iat[i, -1] = 1
            else:
                df.iat[i, -1] = 0
        elif array[[0]] == 1:
            df.iat[i, -1] = 1
        else:
            df.iat[i, -1] = 0
    df = df.drop("Index_Not", axis = 1)
    
    return df

# 4 Final dataframe process

## 4.1 Connect

In [16]:
client = DataFrameClient('localhost', 8086, 'root', 'root')
client.switch_database("SecondM")

In [17]:
client.get_list_measurements()
client.drop_measurement("Dataset")
client.get_list_measurements()

[{'name': 'Book'}, {'name': 'Price'}, {'name': 'difBook'}]

## 4.2 Main core

In [18]:
hours_number = 1
interval = 15

start_time = datetime.strptime("2020-05-21 16:02:30.000000", "%Y-%m-%d %H:%M:%S.%f")
endtime = start_time + timedelta(minutes = interval)

finish_process_time = start_time + timedelta(minutes = 14)

pairs = ["ethbtc"]
# pairs = ["ethbtc", "btcusd", "ethusd", "xtzusd"]

exchanges_snap = ["Binance", "Bitfinex", "Bithumb", "Bitstamp", "Coinbase", "Huobi", "Kraken"]

analysis_start = None

max_dif = 0
time_max_dif = 0

last_rows = [None]*9

last_batch = 0

In [19]:
# Check disconnections!

# TO DO, check disconnections for start and end times. Add field in new table for sequence.
# REMMEMBER TO DELETE THE FIRST ROW BEFORE REMOVING DUPLICATES

# For every hour
start = time.time()

while start_time <= finish_process_time:
        
    last_trades, book_snap, book_dif = query(start_time, endtime)

    # We create the base dataset
    core_df = create_core(last_trades, pairs[0],exchanges_snap,last_rows[7])
    
    last_rows[7] = core_df.iloc[-1,:]
    
    # We add orderbook data
    for k in range(len(exchanges_snap)):
        print(exchanges_snap[k])
        part = book_snap[book_snap["Host"] == exchanges_snap[k]]
        part = part[part["Pair"].str.contains(pairs[0])].loc[:, part.columns != 'Batch_ID']
        part = part[part["Pair"] == pairs[0]]
        
        part_dif = book_dif[book_dif["Host"] == exchanges_snap[k]]
        part_dif = part_dif[part_dif["Pair"].str.contains(pairs[0])].loc[:, part_dif.columns != 'Batch_ID']
        part_dif = part_dif[part_dif["Pair"] == pairs[0]]

        [core_df, last_rows[k]] = create_OrderBook(exchanges_snap[k], core_df, part, part_dif,last_rows[k])


    # Compute last metrics
    core_df = last_metrics_rmdups(core_df)
    
    # Check where the maximum difference is
    if np.max(core_df["Maximum"]) > max_dif:
        max_dif = np.max(core_df["Maximum"]) 
        time_max_dif = start_time
    
    # Create input and calculate labels
    
    columns = [col for col in core_df if (col.startswith('Price_') or col.startswith('Quan_'))]
    columns2 = [col for col in core_df if (col.startswith("Last")  or col.startswith("Q_"))]
    matrixes = np.concatenate([np.asarray(core_df[columns2]).reshape(len(core_df), 2,7).transpose(0, 2, 1),
                   np.asarray(core_df[columns]).reshape(len(core_df), 7,160)], axis = 2)
    core_df["Input"] = [matrixes[i].reshape(7,162) for i in range(len(core_df))]
    core_df = core_df.drop(columns + columns2, axis = 1)
    
    if type(last_rows[8]) != type(None):
        core_df = pd.concat([last_rows[8], core_df])
        
    last_rows[8] =  core_df.iloc[-500::,:]   

    # I think because of the notation 1E-5 influxdb read things as string so we convert everything to float
    
    print(str(datetime.fromtimestamp(time.time())) + ": Writing to dataframe with start time " + str(start_time))
    filter_col = [col for col in core_df if (col.startswith('Maximum') or col.startswith('Ave') or col.startswith('Order'))]
    for col in filter_col:
        core_df.at[:,col] = core_df.loc[:,col].astype(float)
        
    # Create Labels
    core_df = calculate_labels(core_df)
    
    # Write to database
        
    client.write_points(core_df.iloc[0:-500,:].assign(Batch_ID = [i for i in range(last_batch, last_batch+len(core_df)-500)]),
                        "Dataset",time_precision = "n", database = "SecondM", batch_size = 100)
    last_batch = last_batch+len(core_df)-500
    
    # We set up the interval times, and gather the first timestamp that we have data
    start_time = endtime
    endtime = start_time + timedelta(minutes = interval)
    if core_df.isna().sum().sum() == 0 and type(analysis_start) != type(None):
        analysis_start = endtime
    

end = time.time() - start

print("Total time: " + str(datetime.fromtimestamp(end)))

Binance
Bitfinex
Bithumb
Bitstamp
Coinbase
Huobi
Kraken
2020-05-28 23:08:46.828219: Writing to dataframe with start time 2020-05-21 16:02:30
Total time: 1970-01-01 01:06:58.890615


In [20]:
core_df

,OrderBookPrice Binance,OrderBookPrice Bitfinex,OrderBookPrice Bithumb,OrderBookPrice Bitstamp,OrderBookPrice Coinbase,OrderBookPrice Huobi,OrderBookPrice Kraken,Maximum,Average,MaxExch,MinExch,Input,Index
2020-05-21 16:04:47.428157952+00:00,NaN,NaN,NaN,0.022081,NaN,NaN,NaN,0.000000,0.000044,Binance,Bitstamp,"[[nan, 0.0, nan, nan, nan, nan, nan, nan, nan,...",0
2020-05-21 16:04:47.693092096+00:00,NaN,NaN,NaN,0.022081,NaN,NaN,NaN,0.000000,0.000044,Binance,Bitstamp,"[[nan, 0.0, nan, nan, nan, nan, nan, nan, nan,...",0
2020-05-21 16:04:48.021548032+00:00,NaN,NaN,NaN,0.022081,NaN,NaN,NaN,0.000000,0.000044,Binance,Bitstamp,"[[nan, 0.0, nan, nan, nan, nan, nan, nan, nan,...",0
2020-05-21 16:04:48.285448960+00:00,NaN,NaN,NaN,0.022081,NaN,NaN,NaN,0.000000,0.000044,Binance,Bitstamp,"[[nan, 0.0, nan, nan, nan, nan, nan, nan, nan,...",0
2020-05-21 16:04:48.570487040+00:00,NaN,NaN,NaN,0.022081,NaN,NaN,NaN,0.000000,0.000044,Binance,Bitstamp,"[[nan, 0.0, nan, nan, nan, nan, nan, nan, nan,...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-21 16:17:29.814000+00:00,0.022093,0.022085,0.022074,0.022097,0.02208,0.022089,0.022095,0.000023,0.000044,Bithumb,Bitstamp,"[[0.022093500000000002, 0.0, 0.022094, 0.013, ...",0
2020-05-21 16:17:29.827346944+00:00,0.022093,0.022085,0.022074,0.022097,0.02208,0.022089,0.022095,0.000023,0.000044,Bithumb,Bitstamp,"[[0.022093500000000002, 0.0, 0.022094, 0.013, ...",0
2020-05-21 16:17:29.860000+00:00,0.022093,0.022085,0.022074,0.022097,0.02208,0.022089,0.022095,0.000023,0.000044,Bithumb,Bitstamp,"[[0.022099, 0.0, 0.022094, 0.013, 0.022092, 1....",0
2020-05-21 16:17:29.861000+00:00,0.022093,0.022085,0.022074,0.022097,0.02208,0.022089,0.022095,0.000023,0.000044,Bithumb,Bitstamp,"[[0.022099, 0.0, 0.022094, 0.013, 0.022092, 1....",0
